# 02 - Baseline Models

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import os
import sys
import pprint

import matplotlib.pyplot as plt
import matplotlib as mpl

import seaborn as sns
plt.style.use('ggplot')
sns.set()
# plt.style.use('seaborn')
mpl.style.use('seaborn')

pd.options.mode.chained_assignment = None  # default='warn'
pd.options.display.max_columns = 999
pd.options.display.max_rows = 500
np.set_printoptions(precision=6)

DATA_PATH = '../data'
pp = pprint.PrettyPrinter(indent=4)

print(pd.__version__)

# UGLY HACK - not for production
sys.path.append('..')

0.23.4


In [3]:
import utils

TRAIN_FILE = os.path.join(DATA_PATH, 'processed', 'baseline', 'train.csv')
VALID_FILE = os.path.join(DATA_PATH, 'processed', 'baseline', 'val.csv')

train_df = pd.read_csv(TRAIN_FILE)
valid_df = pd.read_csv(VALID_FILE)

print(train_df.shape)
print(valid_df.shape)

FEATURES_FILE = os.path.join(DATA_PATH, 'processed', 'baseline', 'features.json')
features = utils.Features()
features.load(FEATURES_FILE)

# pp.pprint(features)

continuous = features['continuous']
output = features['output']

print(continuous)

(91624, 28)
(1054, 28)
['age', 'time_in_hospital', 'num_lab_procedures', 'num_procedures', 'num_medications', 'number_outpatient', 'number_emergency', 'number_inpatient', 'number_diagnoses']


In [4]:
print(train_df['readmitted'].value_counts())
print(valid_df['readmitted'].value_counts())

1    45812
0    45812
Name: readmitted, dtype: int64
1    527
0    527
Name: readmitted, dtype: int64


In [5]:
X_train = train_df[continuous].values.astype(np.float32)
X_valid = valid_df[continuous].values.astype(np.float32)

X_train = (X_train - X_train.mean(axis=0)) / X_train.std(axis=0)
X_valid = (X_valid - X_train.mean(axis=0)) / X_train.std(axis=0)

y_train = train_df[output].values.astype(np.float32)
y_valid = valid_df[output].values.astype(np.float32)
y_train = y_train.ravel()
y_valid = y_valid.ravel()

# print(np.shape(X_train))
# print(np.shape(y_train))


In [18]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(C=1e5, n_jobs=4, solver='lbfgs')

logreg.fit(X_train, y_train.ravel())

LogisticRegression(C=100000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=4, penalty='l2', random_state=None,
          solver='lbfgs', tol=0.0001, verbose=0, warm_start=False)

In [19]:
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix



acc_train = accuracy_score(y_train, logreg.predict(X_train))
auc_train = roc_auc_score(y_train, logreg.predict_proba(X_train)[:,1])

acc_valid = accuracy_score(y_valid, logreg.predict(X_valid))
auc_valid = roc_auc_score(y_valid, logreg.predict_proba(X_valid)[:,1])

print('Train')
print(f'  Acc: {acc_train}')
print(f'  Auc: {auc_train}')
print(confusion_matrix(y_train, logreg.predict(X_train)))

print('Valid')
print(f'  Acc: {acc_valid}')
print(f'  Auc: {auc_valid}')
print(confusion_matrix(y_valid, logreg.predict(X_valid)))

Train
  Acc: 0.5650921156028988
  Auc: 0.5879194415102174
[[27457 18355]
 [21493 24319]]
Valid
  Acc: 0.5
  Auc: 0.5853115807135734
[[  0 527]
 [  0 527]]
